In [14]:
# import libraries
import pandas as pd
import pygeos
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pathlib
from pathlib import Path
import os
import seaborn as sns
import datetime
import shapely
from shapely.geometry import shape, Point, LineString, Polygon
from collections import defaultdict

# set the working directory
BASE_DIR = Path.cwd()
print(BASE_DIR)

#set the project directory
project_folder = BASE_DIR.parent.parent
print(project_folder)

In [15]:
# Upload output in the respective bucket storage in the following path
# "gs://beam-core-outputs/output/city_name/simulation_name/Output/*" or
# "gs://beam-core-outputs/output/city_name/simulation_name/Output/plot/*" or

from google.cloud import storage
# Upload file to Google Cloud service
def upload_blob(_bucket_name, _source_file_name, _destination_blob_name):
    """Uploads a file to the bucket."""
    # The ID of your GCS bucket
    # bucket_name = "your-bucket-name"
    # The path to your file to upload
    # source_file_name = "local/path/to/file"
    # The ID of your GCS object
    # destination_blob_name = "storage-object-name"
    storage_client = storage.Client()
    bucket = storage_client.bucket(_bucket_name)
    blob = bucket.blob(_destination_blob_name)
    # Optional: set a generation-match precondition to avoid potential race conditions
    # and data corruptions. The request to upload is aborted if the object's
    # generation number does not match your precondition. For a destination
    # object that does not yet exist, set the if_generation_match precondition to 0.
    # If the destination object already exists in your bucket, set instead a
    # generation-match precondition using its generation number.
    generation_match_precondition = 0
    blob.upload_from_filename(_source_file_name, if_generation_match=generation_match_precondition)
    print(
        f"File {_source_file_name} uploaded to {_destination_blob_name}."
    )

def delete_blob(_bucket_name, _blob_name):
    """Deletes a blob from the bucket."""
    # bucket_name = "your-bucket-name"
    # blob_name = "your-object-name"

    storage_client = storage.Client()

    bucket = storage_client.bucket(_bucket_name)
    blob = bucket.blob(_blob_name)
    generation_match_precondition = None

    # Optional: set a generation-match precondition to avoid potential race conditions
    # and data corruptions. The request to delete is aborted if the object's
    # generation number does not match your precondition.
    blob.reload()  # Fetch blob metadata to use in generation_match_precondition.
    generation_match_precondition = blob.generation

    blob.delete(if_generation_match=generation_match_precondition)

    print(f"Blob {_blob_name} deleted.")

In [16]:
# Basic Reference Points
city_name="austin"
gcloud_bucket = "gs://beam-core-outputs/output/austin"
iter_no = "0"
linkstats_file_path = f"{iter_no}.linkstats.csv.gz"
network_file_path = "network.csv.gz"
event_files_path = f"{iter_no}.events.csv.gz"
simulation_name = "austin-base-with-freight-2018__2023-04-14_16-11-13_yoq"
analysis_type = "Freight"
#Check if the folder exist, if not create it.
try:
    pathlib.Path(project_folder.joinpath("Output", city_name, simulation_name, analysis_type, "plot")).mkdir(parents=True, exist_ok=False)
except:
    pass

In [17]:
# RADAR File location
detector_location = gpd.read_file(project_folder.joinpath("Reference_Data", city_name, "RADAR", "detector_location.geojson"))
detector_location = detector_location.to_crs(3857) # set it to PCS instead of GCC

# File location for the beam network created in previous notebook instance run
beam_network = gpd.read_file(project_folder.joinpath("Output", city_name, simulation_name, "Freight", "beam_network_by_county.geojson"))
roadway_type = ['motorway','motorway_link','primary','primary_link','secondary','secondary_link','trunk','trunk_link','tertiary','tertiary_link']
beam_network_with_cars = beam_network.loc[beam_network["attributeOrigType"].isin(roadway_type)].copy()
beam_network_with_cars = beam_network_with_cars.to_crs(3857)

# intersect beam_network_with_cars with austin_npmrds_station, and keep only those links which has distance<50meter
beam_network_with_radar = gpd.sjoin_nearest(beam_network_with_cars, detector_location, distance_col="distance_to_radar_stn", max_distance=50)

beam_network_with_radar = beam_network_with_radar.sort_values(["linkId", "KITS_ID", "distance_to_radar_stn"],ascending=[True, True, True]).drop_duplicates(subset="linkId", keep="first").copy()

In [18]:
fig, ax=plt.subplots(figsize=(30,30))
austin_boundary = gpd.read_file(project_folder.joinpath("Reference_Data", city_name, "Austin_Counties.geojson"))
austin_boundary = austin_boundary.to_crs(3857)
austin_boundary.plot(ax=ax,lw=0.7,column='name', categorical=True, cmap='Pastel2',legend=True, legend_kwds={'fontsize':22,'frameon':False})
beam_network_with_radar.to_crs(3857, inplace=True)
beam_network_with_radar.plot(ax=ax,edgecolor='black',alpha= 0.4,lw=0.7)
detector_location.to_crs(3857, inplace=True)
detector_location.plot(ax=ax,marker="^",markersize=75,alpha= 0.9, c= '#ffff00',)

ax.axis('off')
ax.set_title(f'{city_name.title()} BEAM Network RADAR Screenlines',fontsize=32)
plt.tight_layout()
plt_file_name=f"{city_name.title()}_BEAM_network_Radar_Screenlines.png"
plt.savefig(project_folder.joinpath("Output", city_name, simulation_name, analysis_type, "plot", plt_file_name), dpi=600)

bucket_name = "beam-core-outputs"
source_file_name = project_folder.joinpath("Output", city_name, simulation_name, analysis_type, "plot",plt_file_name)
destination_blob_name = f"output/{city_name}/{simulation_name}/Output/plot/{plt_file_name}"
# Check if the file exist in the bucket. If "Yes", delete
try:
    delete_blob(_bucket_name=bucket_name, _blob_name=destination_blob_name)
except:
    pass
# and upload the file
upload_blob(bucket_name, source_file_name, destination_blob_name)

In [19]:
gpd_fc_file_name = f"{city_name.title() }_BEAM_Network_Radar_Screenlines.geojson"
beam_network_with_radar.to_file(project_folder.joinpath("Output", city_name, simulation_name, analysis_type, gpd_fc_file_name), driver="GeoJSON")

bucket_name = "beam-core-outputs"
source_file_name = project_folder.joinpath("Output", city_name, simulation_name, analysis_type, gpd_fc_file_name)
destination_blob_name = f"output/{city_name}/{simulation_name}/Output/{gpd_fc_file_name}"
# Check if the file exist in the bucket. If "Yes", delete
try:
    delete_blob(_bucket_name=bucket_name, _blob_name=destination_blob_name)
except:
    pass
# and upload the file
upload_blob(bucket_name, source_file_name, destination_blob_name)

In [20]:
df_file_name = f"{city_name.title() }_BEAM_Network_Radar_Screenlines.csv"
beam_network_with_radar.loc[:, ~beam_network_with_radar.columns.isin(["geometry"])].to_csv(project_folder.joinpath("Output", city_name, simulation_name, analysis_type, df_file_name))
bucket_name = "beam-core-outputs"
source_file_name = project_folder.joinpath("Output", city_name, simulation_name, analysis_type, df_file_name)
destination_blob_name = f"output/{city_name}/{simulation_name}/Output/{df_file_name}"
# Check if the file exist in the bucket. If "Yes", delete
try:
    delete_blob(_bucket_name=bucket_name, _blob_name=destination_blob_name)
except:
    pass
# and upload the file
upload_blob(bucket_name, source_file_name, destination_blob_name)

In [21]:
# # intersect beam network with RADAR station featureclass, keep records of only those features which are near to RADAR station location <=50 meter
# dict_linkIds = defaultdict(list)
# dict_radarIds = defaultdict(list)
# for index, row in detector_location.iterrows():
#     if isinstance(row.geometry, Point):
#         beam_network_with_cars['Nearest_Point_distances'] = beam_network_with_cars.distance(row.geometry)
#         lst_links = beam_network_with_cars.loc[beam_network_with_cars["Nearest_Point_distances"]<=50]["linkId"].to_list()
#         for link in lst_links:
#             dict_linkIds.setdefault(link, []).append(row["KITS_ID"])
#             dict_radarIds.setdefault(row["KITS_ID"], []).append(link)
#
# # convert the dictionary into a dataframe
# dfNLinks4RadarStn = pd.concat({key: pd.Series(val) for key, val in dict_linkIds.items()},axis=1).transpose().reset_index()
# # rename column heads
# dfNLinks4RadarStn.rename(columns={"index":"linkId", 0:"KITS_ID"},inplace=True)
# # Update the beam network features with nearest RADAR Station.
# dictLinks = dict(zip(dfNLinks4RadarStn["linkId"], (dfNLinks4RadarStn["KITS_ID"])))
# beam_network_with_radar = beam_network_with_cars.copy()
# beam_network_with_radar["KITS_ID"] = beam_network_with_radar["linkId"].map(dictLinks)

In [22]:
# # plot RADAR Station location
# austin_boundary = gpd.read_file(project_folder.joinpath("Reference_Data", city_name, "Austin_Counties.geojson"))
# austin_boundary.to_crs(4326, inplace=True)
#
# fig, ax=plt.subplots(figsize=(30,30))
# austin_boundary.plot(ax=ax,lw=0.7,column='name', categorical=True, cmap='Pastel2',legend=True, legend_kwds={'fontsize':22,'frameon':False})
# beam_network_with_radar.to_crs(4326, inplace=True)
# beam_network_with_radar.plot(ax=ax,edgecolor='black',alpha= 0.4,)
# detector_location.to_crs(4326, inplace=True)
# detector_location.plot(ax=ax,marker="^",markersize=350,alpha= 0.9, c= '#ffff00',)
#
# ax.axis('off')
# ax.set_title(f'{city_name} RADAR Station location',fontsize=32)
# plt.tight_layout()
# plt_file_name=f"detector_location.png"
# plt.savefig(project_folder.joinpath("Output", city_name, simulation_name, analysis_type, "plot", plt_file_name), dpi=600)

In [23]:
# # export the file as BEAM_network_screenlines.geojson
# selected_beam_network_out = beam_network_with_radar.dropna(subset=["KITS_ID"])
# gpd_fc_file_name = "BEAM_network_screenlines.geojson"
# selected_beam_network_out.to_file(project_folder.joinpath("Output", city_name, simulation_name, analysis_type, gpd_fc_file_name), driver="GeoJSON")
# bucket_name = "beam-core-outputs"
# source_file_name = project_folder.joinpath("Output", city_name, simulation_name, analysis_type, gpd_fc_file_name)
# destination_blob_name = f"output/{city_name}/{simulation_name}/Output/{gpd_fc_file_name}"
# # Check if the file exist in the bucket. If "Yes", delete
# try:
#     delete_blob(_bucket_name=bucket_name, _blob_name=destination_blob_name)
# except:
#     pass
# # and upload the file
# upload_blob(bucket_name, source_file_name, destination_blob_name)
#
# df_file_name = "BEAM_network_screenlines.csv"
# selected_beam_network_out.loc[:, ~selected_beam_network_out.columns.isin(["geometry"])].to_csv(project_folder.joinpath("Output", city_name, simulation_name, analysis_type, df_file_name))
# bucket_name = "beam-core-outputs"
# source_file_name = project_folder.joinpath("Output", city_name, simulation_name, analysis_type, df_file_name)
# destination_blob_name = f"output/{city_name}/{simulation_name}/Output/{df_file_name}"
# # Check if the file exist in the bucket. If "Yes", delete
# try:
#     delete_blob(_bucket_name=bucket_name, _blob_name=destination_blob_name)
# except:
#     pass
# # and upload the file
# upload_blob(bucket_name, source_file_name, destination_blob_name)

In [24]:
# # perform nearest neighbour analysis
# def nearest_radar_point(x):
#     dfRADARStn_geom = x
#      # Calculating distance from the Point to SFChamp line FC
#     beam_network_with_cars['Nearest_Point_distances'] = beam_network_with_cars.distance(dfRADARStn_geom)
#
#     fc_subset = (beam_network_with_cars.loc[beam_network_with_cars['Nearest_Point_distances']
#                  .rank(method='first', ascending=True) <= 5]
#                  .sort_values(by='Nearest_Point_distances', ascending=True))
#     # pid = x["KITS_ID"]
#     # fid = fc_subset.loc[fc_subset["Nearest_Point_distances"]<=5]["linkId"].to_list()
#     # print(f"{pid}: {fid}")
#     return fc_subset.index[fc_subset["Nearest_Point_distances"]<=5].to_list()
#
#
# detector_location["nearest_links"]=""
# dict_linkids = dict()
# for index, row in detector_location.iterrows():
#     if isinstance(row.geometry, Point):
#         dict_linkids[row["KITS_ID"]] = nearest_radar_point(row.geometry)
#         # row["Line_IDs"] = nearest_links(row.geometry)
#         # print(row["KITS_ID"])
#     else:
#         continue


In [25]:
# dict_linkIds = defaultdict(list)
# dict_radarIds = defaultdict(list)
# for index, row in detector_location.iterrows():
#     if isinstance(row.geometry, Point):
#         beam_network_with_cars['Nearest_Point_distances'] = beam_network_with_cars.distance(row.geometry)
#         lst_links = beam_network_with_cars.loc[beam_network_with_cars["Nearest_Point_distances"]<=50]["linkId"].to_list()
#         for link in lst_links:
#             dict_linkIds.setdefault(link, []).append(row["KITS_ID"])
#             dict_radarIds.setdefault(row["KITS_ID"], []).append(link)
#
#         # print(row["KITS_ID"])
#         # print(beam_network_with_cars.loc[beam_network_with_cars["Nearest_Point_distances"]<=50]["linkId"].to_list())


In [26]:
# austin_boundary = gpd.read_file(project_folder.joinpath("Reference_Data", city_name, "Austin_Counties.geojson"))
# austin_boundary.to_crs(4326)
#
# fig, ax=plt.subplots(figsize=(30,30))
# austin_boundary.plot(ax=ax,lw=0.7,column='name', categorical=True, cmap='Spectral',legend=True, legend_kwds={'fontsize':22,'frameon':False})
# beam_network_with_radar.plot(ax=ax,edgecolor='black')
# ax.axis('off')
# ax.set_title(f'{city_name} RADAR detection location',fontsize=32)
# plt.tight_layout()
# plt.savefig(project_folder.joinpath("Output", city_name, simulation_name, "Freight", "plot", "austin_detector_location.png"), dpi=600)